![](https://www.snowflake.com/wp-content/themes/snowflake/assets/img/brand-guidelines/logo-sno-blue-example.svg) 

## 2.1 Viewing the data with a notebook 



The notebook compute comes pre installed with some basic packages which include snowpark and streamlit. In this scenario we would also like to leverage matplotlib. As this package is freely available within the Snowflake Anoconda channel, you can install it easily using the packages dropdown packages, add matplotlib and also add pydeck (this is for the final exercise)

In [ ]:

#  Copyright (c) 2023 Snowflake Computing Inc. All rights reserved.

# Import python packages
import streamlit as st
import pandas as pd
from snowflake.snowpark import functions as F   
from snowflake.snowpark.window import Window
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
from snowflake.snowpark import types as T

## 2.2 THE WHO
![The Who](https://www.gransnet.com/cms/uploads/GN_Images/content/legal-and-money/all_shutterstock_fireplace.jpg)

Our first part of the analysis is to look at the WHO. The provided shared dataset contains a synthetic population dataset. We will have a look at the contents of this.


In [ ]:
population = session.table('COLD_WEATHER_PAYMENTS_DATASET.DATA."Synthetic Population"')

col1,col2,col3,col4= st.columns(4)

with col1:
    st.metric('Total Population: ', population.count())
with col2:
    st.metric('Total Households:', population.select('HOUSEHOLD').distinct().count())
with col3:
    st.metric('Total Not Working', population.filter(F.col('OCCUPATION_CODE')==2).count())
with col4:
    st.metric('Total Under 16yr olds', population.filter(F.col('OCCUPATION_CODE')!=1).count())

You can also view the same information using SQL.

In [ ]:

SELECT COUNT(*) "Total People", APPROX_COUNT_DISTINCT(HOUSEHOLD) "Total Households", COUNT(CASE OCCUPATION_CODE WHEN 2 THEN 1 END) "Total Not Working" FROM COLD_WEATHER_PAYMENTS_DATASET.DATA."Synthetic Population"

Now lets look at a sample of the population. We will look at a sample of 20% of the population and then limit the return to 100 rows

In [ ]:

population.sample(0.2).limit(100);

Lets see counts of the population by occupations and gender

In [ ]:

gender = population.group_by('SEX').count()
occupation = population.group_by('OCCUPATION').agg(F.any_value('OCCUPATION_CODE').alias('Occupation Code')
                                                   ,F.count('*').alias('COUNT'))

st.table(gender)
st.table(occupation)

We will utilise streamlit's basic charting capabilities to simply look at the distribution by occupation and gender

In [ ]:

st.markdown('People by Occupation and Sex')
col1, col2 = st.columns(2)
with col1:
    st.bar_chart(occupation,x='OCCUPATION',y='COUNT')
with col2:
    st.bar_chart(gender,x='SEX',y='COUNT')


We can use this information to filter the citizens

In [ ]:

col1,col2,col3 = st.columns(3)
with col1:
    Gender = st.radio('Gender',gender)
with col2:
    elderly = st.selectbox('Occupation',occupation)
with col3:
    Age_Range = st.slider('Age Range',1,99,(1,99))


Add a SQL sell which will reveal a sample of the sample population. The parameters you have just created will be used to filter the query below.

In [ ]:

select * from (select * from COLD_WEATHER_PAYMENTS_DATASET.DATA."Synthetic Population"  where SEX = '{{Gender}}' and AGE BETWEEN {{Age_Range[0]}}AND {{Age_Range[1]}} )sample(100 rows)


For the calculator, I have decided that all policies will be based around citizens who are not working, and live in households where everyone else is not working.  In reality, the selection will be based on numerous factors.

Lets start of by creating a dataset based on people who are not working.

In [ ]:

population_not_working = population.filter(F.col('OCCUPATION_CODE')==2)

population_not_working.limit(10)


We will now create a table which counts the number of people working in every household.

In [ ]:

population_working = population.filter((F.col('OCCUPATION_CODE')!=2) | (F.col('OCCUPATION_CODE')==1))

working_household = population_working.select('HOUSEHOLD','NI NUMBER').group_by(F.col('HOUSEHOLD')).agg(F.count('*').alias('WORKING_PEOPLE'))

working_household.limit(10)


Let's now visualise the people who are not working and also do not live with anyone who is working. To do this we did a join to the the working household dataframe we just created and then filtered out any matches. We are also importing matplotlib to visualise the distribution of key metrics.

In [ ]:

import matplotlib.pyplot as plt

population_entitled_cold_weather = population_not_working.join(working_household, on=(population_not_working['HOUSEHOLD']==working_household['HOUSEHOLD']), how='outer',rsuffix='_L').drop('HOUSEHOLD_L')\
.filter(F.col('WORKING_PEOPLE').isNull()).drop('WORKING_PEOPLE')

st.metric('Total entitled for cold weather payments:', population_entitled_cold_weather.count())

st.markdown('#### Sample of data extracted')
hist_sample = population_entitled_cold_weather.sample(0.2)#.limit(1000)
hist = hist_sample.select(F.col('AGE'),'MORBILITIES','YEARS_AT_ADDRESS','DISTANCE_FROM_PRACTICE').to_pandas().hist(bins=7)

col1,col2,col3 = st.columns([0.2,0.6,0.2])
with col2:
    plt.show()


Now, let's create a table with names and addresses of all households who will get a cold weather payment if the weather permits this.

In [ ]:

households_cold_weather = population_entitled_cold_weather.with_column('ELECTRICITY_BILL_PAYER',F.concat('FIRST_NAME',F.lit(' '),'LAST_NAME')).group_by('HOUSEHOLD','ADDRESS_1','ADDRESS_2','ADDRESS_3','POSTCODE','LSOA_CODE')\
.agg(F.any_value('ELECTRICITY_BILL_PAYER').alias('HOUSEHOLD_BILL_PAYER'),F.count('*').alias('NUMBER OF OCCUPANTS'))

households_cold_weather.sample(0.2).limit(10)


We have now managed to work out who would be entitled based on who is not working, and who doesn't live with anyone who is working. Of course, in reality the selection would be more scientific - such as measuring based on who is receiving universal credits.

In [ ]:

postcodes = session.table('RESIDENTIAL_POSTCODES.GEOLOCAL.GEOLOCAL_RESIDENTIAL_POSTCODE')
postcodes = postcodes.select('"PCD"',F.col('LAT').astype(T.FloatType()).alias('LAT'),F.col('LON').astype(T.FloatType()).alias('LON'))
postcodes.limit(10)


## 2.3 THE WHERE
![The Where](https://camo.githubusercontent.com/c8d321142c12bbbe569fe59237bd6c62faa7f6c0fdc29109fda223c6bf06e0cd/68747470733a2f2f656e637279707465642d74626e302e677374617469632e636f6d2f696d616765733f713d74626e3a414e6439476354784a4c344e74527a657665314d52744e614b626331464f35514333762d6d7a6f426d412673)

In order to understand the where, we need to look at the location of the residents. We have postcodes but we do not currently know whereabouts in the world they are linked to. The More Metrics dataset has a free listing of all UK postcodes.

Lets now join these postcodes to the households who may be entitled to cold weather payments.

In [ ]:
households_cold_weather_with_points = postcodes.join(households_cold_weather,type='inner',
                     on=postcodes['"PCD"']==households_cold_weather['POSTCODE'])


We will now leverage the streamlit module st.map to visualise where the residents are located.

In [ ]:
sample = households_cold_weather_with_points.sample(0.01)

st.map(sample)
st.dataframe(sample)

## 2.4 The When
![The When](https://www.telegraph.co.uk/multimedia/archive/03478/bttf2_3478174a.jpg?imwidth=1280)

We want the policy to pay a cold weather payment only when the weather has reached a certain level. At this point in time, its based on postcode, and its based on if the weather gets colder than 0 degrees in any 7 day rolling period. For this calculation, we need historical weather data. This is what we will use the met office weather data for.

In [ ]:

summary_data = session.table('COLD_WEATHER_PAYMENTS_DATASET.DATA."Hourly Forecast"')
summary_data.show()


Creating the calculation will require time series analysis. Lets construct a date from the 'Valid Hour' column and filter the dates to be when the policy is valid.

In [ ]:

hourly_with_date = summary_data.with_column('"Date"',
                         F.date_from_parts(F.substr('"Valid Hour"',1,4),
                                          F.substr('"Valid Hour"',5,2),
                                          F.substr('"Valid Hour"',7,2)))

hourly_with_date_grp = hourly_with_date.filter(F.col('"Date"').between('2022-11-01','2023-03-31'))\
.group_by('"Date"').agg(F.avg(F.cast('"Instantaneous Screen Temperature"',T.FloatType())).alias('Instantaneous Screen Temperature'))


Let's look at at Screen Temperature, to see what impact that may have.

In [ ]:

st.line_chart(hourly_with_date_grp,y='Instantaneous Screen Temperature',x='Date')


We will then group the average temperature by the weather station and date - we want to see average temperature per day rather than hourly.

In [ ]:

hourly_with_date = hourly_with_date.groupBy(F.col('"SSPA Identifier"'),
                         F.col('"Date"')).agg(F.avg('"Instantaneous Screen Temperature"').alias('AVERAGE_TEMP'))

hourly_with_date.limit(10)


You will note that the where is in fact a site identifier. We want to change this so we have postcode sector instead. A mapping table is used to map the site with postcode.

In [ ]:

weather_station = session.table('COLD_WEATHER_PAYMENTS_DATASET.DATA.PCSECTORMAPPING')\
.select('"SiteID"','PC_SECT','LONG','LAT')\
.with_column('Postcode_Area',F.call_function('SPLIT_PART',F.col('PC_SECT'),'_',1)).distinct()
weather_station.limit(100).to_pandas()


Now we have our mapping, we need to summarize the weather by postcode area (the policy goes by postcode area - i.e (DY13)).

In [ ]:

hourly_with_date_ws = hourly_with_date.join(weather_station,on=weather_station['"SiteID"']==hourly_with_date['"SSPA Identifier"'])\
.group_by('"Date"',
          'POSTCODE_AREA').agg(F.avg(F.cast('LAT',T.FloatType())).alias('LAT'),
                               F.avg(F.cast('LONG',T.FloatType())).alias('LON'),
                               F.avg(F.cast('AVERAGE_TEMP',T.FloatType())).alias('AVERAGE_TEMP'))

hourly_with_date_ws.limit(10)


Because we need the calculation to be based on a moving average, we need the next calculation to be dynamic. Snowflake supports window functions - which allows the calculation to be applied after the result set is generated.

[More info on window calculations](https://docs.snowflake.com/en/sql-reference/functions-analytic)

Lets create a python function to calculate the moving average

In [ ]:

def movaverage(days,df):
    window = Window.partition_by(F.col('"POSTCODE_AREA"')).orderBy(F.col('"Date"').desc()).rows_between(Window.currentRow,7)

    # Add moving averages columns for Cloud Cover and Solar Energy based on the previously defined window
    df = df.with_column('"Temp_Max_Temp_7_Days"',F.max(F.cast("AVERAGE_TEMP",T.FloatType())).over(window)).sort('"Date"')
    
    # Change the data type to a float
    df = df.with_column('"AVERAGE_TEMP"',F.cast('"AVERAGE_TEMP"',T.FloatType()))
    
    return df


Let's now apply the moving average function in order to filter our weather to only provide postcodes where the temperature has ben 0 or below for 7 or more consecutive days.

In [ ]:

mov_average = movaverage(7,hourly_with_date_ws).filter(F.col('"Temp_Max_Temp_7_Days"')<=0)
mov_average


We will now join this filtered weather data set to the effected households that would be entitled to a cold weather payment.

In [ ]:

people_affected = mov_average.join(households_cold_weather_with_points.drop('LAT','LON'),
                 on= mov_average['POSTCODE_AREA'] == F.call_function('SPLIT_PART', households_cold_weather_with_points['"PCD"'],F.lit(' '),1))

people_affected


Finally lets view this on a map.

In [ ]:

st.map(people_affected)


![](https://www.snowflake.com/wp-content/themes/snowflake/assets/img/brand-guidelines/logo-sno-blue-example.svg)

So in summary we have looked at some techniques to understand the who, the when and the where.  **Congratulations** you have now finnished this part of the lab.  Please go back to the main guide for section 3